<a href="https://colab.research.google.com/github/HausReport/MightyLogic/blob/master/Notebooks/Turf_War_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hello!

If you're seeing this on GitHub, you can open the document on Google by clicking the "Open in Collab" button above.  Once this document is open in Google, you can see the interactive content by selecting `Runtime` on the menubar and then `Run All.`


# Set up environment


In [32]:
import sys
# need newer plotly/plotly_express than collab provides?
#!{sys.executable} -m pip install --force-reinstall --upgrade "plotly"
#!{sys.executable} -m pip install --force-reinstall --upgrade "plotly_express"

#
# Moving some of the guts to GitHub and refactoring 
#
!{sys.executable} -m pip install --upgrade --force-reinstall "git+https://github.com/HausReport/MightyLogic.git#egg=MightyLogic"

#
# Imports
#
from random import randrange
from scipy.signal import convolve2d
from enum import Enum, auto
from pprint import pprint
import plotly.figure_factory as ff

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import math

#import sklearn
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
import numpy as np
from sklearn.preprocessing import StandardScaler, scale

from MightyLogic.Currency import Currency as Currency
from MightyLogic.Values import Values as Values
from MightyLogic.TurfWar.Tiles.BuildingFactory import *
from MightyLogic.TurfWar.TurfWarMap import TurfWarMap
from MightyLogic.TurfWar.Tiles.TileList import TileList
from MightyLogic.Player import *
from MightyLogic.Player.NightTerror import getNightTerrorPlayerGroup

#Values.GOLD

  Cloning https://github.com/HausReport/MightyLogic.git to /tmp/pip-install-nf05w2av/mightylogic_56a3077ac75d4aa9a45c2c4f33733a5b
  Running command git clone -q https://github.com/HausReport/MightyLogic.git /tmp/pip-install-nf05w2av/mightylogic_56a3077ac75d4aa9a45c2c4f33733a5b
  Using cached pytest-6.2.4-py3-none-any.whl (280 kB)
  Using cached py-1.10.0-py2.py3-none-any.whl (97 kB)
  Using cached iniconfig-1.1.1-py2.py3-none-any.whl (5.0 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached attrs-21.2.0-py2.py3-none-any.whl (53 kB)
  Using cached importlib_metadata-4.6.3-py3-none-any.whl (17 kB)
  Using cached pluggy-0.13.1-py2.py3-none-any.whl (18 kB)
  Using cached packaging-21.0-py3-none-any.whl (40 kB)
  Using cached zipp-3.5.0-py3-none-any.whl (5.7 kB)
  Using cached typing_extensions-3.10.0.0-py3-none-any.whl (26 kB)
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Created wheel for MightyLogic: filename=MightyLogic-0.2-py3-none-any.whl size=266

# Turf War Demo

In [33]:

#
# A Turf War Map
#

map2 =TurfWarMap()
map2.setGuild("Nightterror")
map2.setStartDate(year=2021, month=8, day=2)


# Use convenience functions to build value array
map2.addTile('A',1, gold_mine(50000,400,5000))
map2.addTile('A',2, forest(3000,0,2000))
map2.addTile('A',3, plains(3000, 2000))
map2.addTile('A',4, mountains(3000,2000))
map2.addTile('A',5, forest(3000,0,2000))
map2.addTile('A',6, plains(3000,2000))

map2.addTile('B',1, plains(3000,2000))
map2.addTile('B',2, tree_of_life(3500,600,7000))
map2.addTile('B',3, forest(5000,0,5000))
map2.addTile('B',4, stronghold(600,800,8500))
map2.addTile('B',5, mountains(10000,0))
map2.addTile('B',6, gold_mine(90000,500,5000))

map2.addTile('C',1, forest(3000,0,2000))
map2.addTile('C',2, plains(5000,5000))
map2.addTile('C',3, mountains(10000,0))
map2.addTile('C',4, forest(5000,0,5000))
map2.addTile('C',5, plains(5000,5000))
map2.addTile('C',6, mountains(3000,2000))

map2.addTile('D',1, mountains(3000,2000))
map2.addTile('D',2, lighthouse(2000,400,5000))
map2.addTile('D',3, plains(5000,5000))
map2.addTile('D',4, lighthouse(3500,600,000))
map2.addTile('D',5, forest(5000,0,5000))
map2.addTile('D',6, plains(3000,2000))

map2.addTile('E',1, plains(3000,2000))
map2.addTile('E',2, mountains(3000,2000))
map2.addTile('E',3, factory(1650,500,5000))
map2.addTile('E',4, plains(1600,1000))
map2.addTile('E',5, plains(3000,2000))
map2.addTile('E',6, factory(900,400,5000))


# Payout array
image = map2.getValues()
res = map2.stagingScores()
#pprint(image)

In [34]:
#
# An approach that shows well-labeled heatmaps.
#

def get_heatmap(arr, title, round=True):
  flip = np.flip( np.array(arr), axis=0)  # use numpy to flip array vertically
  z = flip
  if round:
    z = z.round()

  x = ['1', '2', '3', '4','5','6']
  y = ['E','D','C','B','A']

  #z_text = ["%.0f" % number for number in z] - doesn't work

  if round:
    z_text = flip.round( decimals=-1)
  else:
    z_text = flip.round( decimals=3)
  # Possible to build up z_text to make it more friendly
  # z_text = [['Win', 'Lose', 'Win'],
  #          ['Lose', 'Lose', 'Win'],
  #          ['Win', 'Win', 'Lose']]

  fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')
  fig.update_layout(title=title, autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
  return fig #fig.show()



In [35]:
#
# The payout array can be accessed as a Pandas dataframe.
#
payout = map2.getDataFrame()
payout

,1,2,3,4,5,6
A,111000.0,30003.0,30003.0,30003.0,30003.0,30003.0
B,30003.0,114035.0,75005.0,139560.0,10.0,136500.0
C,30003.0,75005.0,10.0,75005.0,75005.0,30003.0
D,30003.0,81020.0,75005.0,9035.0,75005.0,30003.0
E,30003.0,30003.0,82582.5,15001.6,30003.0,81045.0


In [36]:
#
# The staging array can be accessed as a Pandas dataframe.
#
staging =map2.getStrategicDataFrame()
staging

,1,2,3,4,5,6
A,0.0,255038.0,135011.0,199566.0,60016.0,166503.0
B,255038.0,0.0,283608.0,0.0,381068.0,0.0
C,135011.0,225068.0,300020.0,223610.0,180023.0,241508.0
D,141026.0,0.0,172647.5,0.0,144046.0,186053.0
E,60006.0,193605.5,0.0,121620.5,171051.6,0.0


In [37]:
#
# Heatmaps show the best tiles for payout round and non-payout rounds.
#


fig1 = get_heatmap(image,"Payout Values for 3rd Rounds")
fig2 = get_heatmap(res,"Strategic Values for Non-Payout Rounds")

fig1.show()
fig2.show()

In [38]:
#
# Shows payout shares for holding a tile on round 3n 
#

gm = map2.getTile('B',6) #b6 # c5 # gold_mine(90000,500,5000)
print(gm.payouts())


Payouts for B:6 Gold Mine
Place               Reward
First               6,300 gold, 350 contribution
2nd                 5,400 gold, 300 contribution
3rd                 4,500 gold, 250 contribution
4-6                 3,600 gold, 200 contribution
7-11                2,700 gold, 150 contribution
12-25               1,800 gold, 100 contribution
26-44               900 gold, 50 contribution
500 guild influence


In [39]:
#
# List of buildings easy for players to read.
#
map2.printBuildingList(double=False)

Building          Points      For 1st Place                      Guild Gets
B-4: Stronghold   139,560      42 epic souls, 595 contribution,  800 influence.
B-6: Gold Mine    136,500         6,300 gold, 350 contribution,  500 influence.
B-2: Tree of Life  114,035     245 rare souls, 490 contribution,  600 influence.
A-1: Gold Mine    111,000         3,500 gold, 350 contribution,  400 influence.
E-3: Factory       82,582      116 soul dust, 350 contribution,  500 influence.
E-6: Factory       81,045       63 soul dust, 350 contribution,  400 influence.
D-2: Lighthouse    81,020     140 rare souls, 350 contribution,  400 influence.
D-4: Lighthouse     9,035                       245 rare souls,  600 influence.


In [40]:
#
# Text report that highlights building returns 
#

map2.printReport()

Turf War Map Report
Guild: Nightterror
For the Week of: 02 August 2021
Total Map Value: 1,609,868 common souls

Building          Points      For 1st Place                      Guild Gets
B-4: Stronghold   139,560      42 epic souls, 595 contribution,  800 influence.
B-6: Gold Mine    136,500         6,300 gold, 350 contribution,  500 influence.
B-2: Tree of Life  114,035     245 rare souls, 490 contribution,  600 influence.
A-1: Gold Mine    111,000         3,500 gold, 350 contribution,  400 influence.
E-3: Factory       82,582      116 soul dust, 350 contribution,  500 influence.
E-6: Factory       81,045       63 soul dust, 350 contribution,  400 influence.
D-2: Lighthouse    81,020     140 rare souls, 350 contribution,  400 influence.
D-4: Lighthouse     9,035                       245 rare souls,  600 influence.


In [41]:
#
# Evaluate neighbors of a given tile
#
map2.printNeighborList('B',2)

Neighbors of tile B:2
Building          Points      For 1st Place                      Guild Gets
B-1: Plains        30,003   210 common souls, 140 contribution,    0 influence.
B-3: Forest        75,005   350 common souls, 350 contribution,    0 influence.
A-2: Forest        30,003   210 common souls, 140 contribution,    0 influence.
C-2: Plains        75,005   350 common souls, 350 contribution,    0 influence.


# Strategy & Payout Arrays

In [42]:
#
# High values are desirable payout locations, low are less desirable
#

#pd.DataFrame(map2.getScaledPayout())

In [43]:
#
# High values are desirable staging locations, low are less desirable
#

#pd.DataFrame(map2.getScaledStaging())

In [44]:
#
# Lowest values are least desirable staging locations, higher are better staging locations, highest are desirable payout locations
#
adjusted_move = map2.getAdjustedMoves()
pd.DataFrame(adjusted_move)

,0,1,2,3,4,5
0,2.700772,0.561644,0.555385,0.555385,0.555385,0.555385
1,0.561644,2.781161,1.747364,3.457248,0.957315,3.376197
2,0.555385,1.747364,0.702865,1.747364,1.747364,0.555385
3,0.555385,1.906685,1.747364,0.000000,1.747364,0.555385
4,0.555385,0.555385,1.948071,0.158039,0.555385,1.907347


In [45]:
moves = adjusted_move
get_heatmap(moves,"Movement Array", round=False).show()

In [46]:
#
# 3D Presentation of the movement surface
#
flip = np.flip( moves, axis=1) 
fig = go.Figure(data=[go.Surface(z=flip)])
fig.update_layout(title='Movement Array in 3D', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [47]:
retx, rety = map2.getRecommendedMovesArrays()

In [48]:
def getTileTrace():

  # Generate tile name array
  cnames = []
  rtl = ['E','D','C','B','A']
  ctli = [1,2,3,4,5,6]
  for c in ctli:
    for r in rtl:
      cnames.append(f"{r}{c}")
      
  return go.Scatter(x=[0,0,0,0,0,1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,4,4,4,4,4,5,5,5,5,5], y=[0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4],
                          mode='markers+text',
                          textposition="top right",
                          text = cnames,
                          textfont=dict(
                            family="sans serif",
                            size=18,
                            color="red"
                          ),
                          marker_size=14,
                          name='Tiles')
  
def getContourTrace(data):
  z = np.flip( data, axis=0)
  return(go.Contour(
        z=z,
        x=[0,1,2,3,4,5], # horizontal axis
        y=[0,1,2,3,4],   # vertical axis
        colorscale="delta"
        #showlegend=False
        #color_continuous_scale=px.colors.sequential.Viridis
    ))

def squaresTrace():
  pass
  #fig.add_shape(type="rect",
  #  xref="x", yref="y",
  #  x0=-0.5, y0=-0.5,
  #  x1=0.5, y1=0.5,
  #  line=dict(
  #      color="RoyalBlue",
  #      width=3,
  #  ),
  #  #fillcolor="LightSkyBlue",
  # #)

In [49]:
#
# Show staging values as a contour plot
#
fig = go.Figure()
fig.add_trace( getContourTrace(map2.getStrategicDataFrame().to_numpy()) )
fig.add_trace( getTileTrace())
fig.update_layout(title='Strategic Value of Staging Tiles')
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.update_layout(dict(paper_bgcolor='White',plot_bgcolor='White', showlegend=False))
fig.show()

In [50]:
  #fig = ff.create_streamline(x, y, u,v, arrow_scale=.1, name='streamline')
  x,y = np.meshgrid(np.arange(0, 6, 1), np.arange(0, 5, 1))
  fig = ff.create_quiver(x,y,                       #positions 
                         retx, rety,                #vectors
                         scale=1,                   #vector scale
                         name="Recommended Move",   #legend title
                         line=dict(width=2,
                                color='red'),
                        )

  # add dots at integers
  fig.add_trace( getTileTrace())

  # add contour plot   
  z = payout.to_numpy()
  fig.add_trace( getContourTrace(z) )
  
  # cosmetics
  #  - axes are confusing & extraneous
  fig.update_xaxes(visible=False)
  fig.update_yaxes(visible=False)

  #  - KISS   
  showLegend = False #dots & arrows  # 3 legends are colliding
  fig.update_layout(dict(paper_bgcolor='White',plot_bgcolor='White', showlegend=showLegend))

  #fig.update_layout dict(show_legend=False))
  #fig.update_layout(coloraxis_showscale=False)

  # move colorbar
  #fig.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=0,
  #                                        ticks="outside",
  #                                        ticksuffix=" bills"))

  # - graph title
  fig.update_layout(title='Relative Payout and Recommended Moves')
  fig.show()

# Multi-Tile Targets

In [51]:
import re
s = 'a1a5c6e3'
res = re.findall('(\d+|[A-Za-z]+)', s)
print(res)

def parse_tile(inp):
  pass

['a', '1', 'a', '5', 'c', '6', 'e', '3']


In [52]:
image = map2.getValues()


#pprint(image)
filter_kernel =  [[0, 0, 0],
                  [0, 1, 1],
                  [0, 1, 0]]

def getComboValues(filter_kernel):
  filter_kernel = np.flipud(np.fliplr(filter_kernel))
  res = convolve2d(image,
                 filter_kernel,
                 mode='same',
                 boundary='fill',
                 fillvalue=0)
  return res

#pprint(res)
#fig1 = get_heatmap(image,"Payout Values for 3rd Rounds")
#fig1.show()

filter_kernel0 =  [[0, 0, 0],
                  [0, 1, 1],
                  [0, 1, 0]]
drcombos = getComboValues(filter_kernel)

filter_kernel1 =  [[0, 1, 0],
                  [0, 1, 1],
                  [0, 0, 0]]
urcombos = getComboValues(filter_kernel)

filter_kernel2 =  [[0, 0, 0],
                  [1, 1, 0],
                  [0, 1, 0]]
dlcombos = getComboValues(filter_kernel)

filter_kernel3 =  [[0, 1, 0],
                  [1, 1, 0],
                  [0, 0, 0]]
ulcombos = getComboValues(filter_kernel)

filter_kernel4 =  [[0, 0, 0],
                  [1, 1, 1],
                  [0, 0, 0]]
hcombos = getComboValues(filter_kernel)

filter_kernel5 =  [[0, 1, 0],
                  [0, 1, 0],
                  [0, 1, 0]]
vcombos = getComboValues(filter_kernel)

filter_kernels = [filter_kernel0,filter_kernel1,filter_kernel2,filter_kernel3,filter_kernel4,filter_kernel5]
#get_heatmap(drcombos,"down-right combos  ", round=False)
get_heatmap(urcombos,"up-right combos  ", round=False)
get_heatmap(hcombos,"horiz combos  ", round=False)
get_heatmap(vcombos,"vert combos  ", round=False)

#drlcombos.show()

In [53]:

# given row in [0-4] and col in [0-5], return array of valid tile locations
# matching the given convolution matrix

def getFilterTileLocations(r, c, filter_kernel):
  row_dict = {0:'A', 1:'B', 2: 'C', 3:'D', 4: 'E'}
  row=row_dict[r]
  j=c+1

  uprow = map2.upRow(row)
  leftcol = map2.leftCol(j)
  rightcol = map2.rightCol(j)
  dnrow = map2.downRow(row)

  ret = []
  print(uprow)
  # above row
  if uprow is not None:
    if leftcol is not None:
      if filter_kernel[0][0]==1:
        #print(uprow + str(leftcol))
        ret.append([uprow, leftcol])
    if filter_kernel[0][1]==1:
      #print(uprow + str(j))
      ret.append([uprow, j])
    if rightcol is not None:
      if filter_kernel[0][2]==1:
        #print(uprow + str(rightcol))
        ret.append([uprow, rightcol])

  # this row
  if leftcol is not None:
    if filter_kernel[1][0]==1:
      #print(row + str(leftcol))
      ret.append([row, leftcol])
  if filter_kernel[1][1]==1:
    #print(row + str(j))
    ret.append([row, j])
  if rightcol is not None:
    if filter_kernel[1][2]==1:
      #print(row + str(rightcol))
      ret.append([row, rightcol])

  # below row
  if dnrow is not None:
    if leftcol is not None:
      if filter_kernel[2][0]==1:
        #print(dnrow + str(leftcol))
        ret.append([dnrow, leftcol])
    if filter_kernel[2][1]==1:
      #print(dnrow + str(j))
      ret.append([dnrow, j])
    if rightcol is not None:
      if filter_kernel[2][2]==1:
        #print(dnrow + str(rightcol))
        ret.append([dnrow, rightcol])
  return ret

ret = getFilterTileLocations(0, 0, filter_kernel)
pprint(ret)
tarr = map2.getTiles(ret)
tlist = TileList(map2,ret)
#tlist.share(0.07)

None
[['A', 1], ['A', 2], ['B', 1]]


In [54]:
allCombos = []
for filter in filter_kernels:
  for r in [0,1,2,3,4]:
    for c in [0,1,2,3,4,5]:
      allCombos.append(TileList(map2, getFilterTileLocations(r, c, filter_kernel)))

allCombos = set(allCombos)

None
None
None
None
None
None
A
A
A
A
A
A
B
B
B
B
B
B
C
C
C
C
C
C
D
D
D
D
D
D
None
None
None
None
None
None
A
A
A
A
A
A
B
B
B
B
B
B
C
C
C
C
C
C
D
D
D
D
D
D
None
None
None
None
None
None
A
A
A
A
A
A
B
B
B
B
B
B
C
C
C
C
C
C
D
D
D
D
D
D
None
None
None
None
None
None
A
A
A
A
A
A
B
B
B
B
B
B
C
C
C
C
C
C
D
D
D
D
D
D
None
None
None
None
None
None
A
A
A
A
A
A
B
B
B
B
B
B
C
C
C
C
C
C
D
D
D
D
D
D
None
None
None
None
None
None
A
A
A
A
A
A
B
B
B
B
B
B
C
C
C
C
C
C
D
D
D
D
D
D


In [55]:
allCombos = list(allCombos)
allCombos.sort(key=lambda x: x.value, reverse=True)
for combo in allCombos:
  val = combo.getValue()
  print(combo.getTileLine()) #str(val) + " - " + combo.getName())

B:2, B:3, C:2:  264,045  700 common souls, 245 rare souls, 1,190 contribution,  600 influence.
B:3, B:4, C:3:  214,575  1,050 common souls, 42 epic souls, 945 contribution,  800 influence.
B:4, B:5, C:4:  214,575  1,050 common souls, 42 epic souls, 945 contribution,  800 influence.
B:5, B:6, C:5:  211,515  1,050 common souls, 6,300 gold, 700 contribution,  500 influence.
A:4, A:5, B:4:  199,566  420 common souls, 42 epic souls, 875 contribution,  800 influence.
D:2, D:3, E:2:  186,028  560 common souls, 140 rare souls, 840 contribution,  400 influence.
C:5, C:6, D:5:  180,013   910 common souls, 840 contribution,    0 influence.
A:2, A:3, B:2:  174,041  420 common souls, 245 rare souls, 770 contribution,  600 influence.
B:1, B:2, C:1:  174,041  420 common souls, 245 rare souls, 770 contribution,  600 influence.
A:1, A:2, B:1:  171,006  420 common souls, 3,500 gold, 630 contribution,  400 influence.
D:3, D:4, E:3:  166,622  350 common souls, 245 rare souls, 116 soul dust, 700 contributi

In [56]:

topTen = allCombos[:10]
print("Top 3-Tile Combos on Map")
print("Tiles           Points   Reward for first place")
print("--------------------------------------------------------------------------------------------------------")
for combo in topTen:
  print(combo.getTileLine())

Top 3-Tile Combos on Map
Tiles           Points   Reward for first place
--------------------------------------------------------------------------------------------------------
B:2, B:3, C:2:  264,045  700 common souls, 245 rare souls, 1,190 contribution,  600 influence.
B:3, B:4, C:3:  214,575  1,050 common souls, 42 epic souls, 945 contribution,  800 influence.
B:4, B:5, C:4:  214,575  1,050 common souls, 42 epic souls, 945 contribution,  800 influence.
B:5, B:6, C:5:  211,515  1,050 common souls, 6,300 gold, 700 contribution,  500 influence.
A:4, A:5, B:4:  199,566  420 common souls, 42 epic souls, 875 contribution,  800 influence.
D:2, D:3, E:2:  186,028  560 common souls, 140 rare souls, 840 contribution,  400 influence.
C:5, C:6, D:5:  180,013   910 common souls, 840 contribution,    0 influence.
A:2, A:3, B:2:  174,041  420 common souls, 245 rare souls, 770 contribution,  600 influence.
B:1, B:2, C:1:  174,041  420 common souls, 245 rare souls, 770 contribution,  600 influence.

In [57]:
best = TileList(map2, [ ['A', 3],['B', 1],['B', 2], ['B',3], ['B', 4], ['C',1],  ['C', 2],['C', 3], ['D',1]])
print(best.payouts(double=True))


Payouts for Plains at A:3, Plains at B:1, Tree of Life at B:2, Forest at B:3, Stronghold at B:4, Forest at C:1, Plains at C:2, Mountains at C:3, Mountains at D:1 (Sunday Double)
Place               Reward
First               4,480 common souls, 490 rare souls, 84 epic souls, 4,690 contribution
2nd                 3,840 common souls, 420 rare souls, 72 epic souls, 4,020 contribution
3rd                 3,200 common souls, 350 rare souls, 60 epic souls, 3,350 contribution
4-6                 2,560 common souls, 280 rare souls, 48 epic souls, 2,680 contribution
7-11                1,920 common souls, 210 rare souls, 36 epic souls, 2,010 contribution
12-25               1,280 common souls, 140 rare souls, 24 epic souls, 1,340 contribution
26-44               640 common souls, 70 rare souls, 12 epic souls, 670 contribution



In [58]:


desired = [300000,300000,300000]
print(desired)


total = []


players = getNightTerrorPlayerGroup()




[300000, 300000, 300000]


In [59]:
groups = players.splitDesired(desired)
for group in groups:
  group.print()

Report Generated at 19:09 GMT on 02 August 2021
  66,223 Rags
  54,463 Panda
  42,872 Flowbites
  41,480 Erlaed
  32,978 Eragon
  31,438 Wheeler
  29,080 iahobo
  26,869 RoMiC
  25,681 Iceman
  19,765 Seph
  12,275 Nikki
Team troops: 383124.0
Report Generated at 19:09 GMT on 02 August 2021
  60,345 BilBisqqit
  57,040 Hashem
  46,693 LordOtto
  38,869 Sente
  32,983 Stalguard
  31,453 OmniMan
  31,410 ThaBozz
  26,609 Ender
  26,101 Hung
  14,642 Ramzinator
  14,587 Holymight
Team troops: 380732.0
Report Generated at 19:09 GMT on 02 August 2021
  58,747 Malbad
  57,447 Jman
  51,683 chingcheng
  34,719 Karpet
  33,060 negro
  32,337 FriarKen
  29,960 Vasilich
  27,047 PiXeL1K
  26,210 Tech34
  19,690 Beeznutz
Team troops: 370900.0
Report Generated at 19:09 GMT on 02 August 2021
Team troops: 0


In [60]:
names = []
troops = []
for player in players.players:
  names.append(player.name)
  troops.append(player.troops)
  #print(f"{player.name}, {player.troops}")
d = {'Name':names,'Troops':troops}

df = pd.DataFrame(d)
df

,Name,Troops
0,Rags,66223
1,BilBisqqit,60345
2,Malbad,58747
3,Jman,57447
4,Hashem,57040
5,Panda,54463
6,chingcheng,51683
7,LordOtto,46693
8,Flowbites,42872
9,Erlaed,41480


In [61]:
fig = px.histogram(data_frame=df, x=troops, 
                   title="Number of Troops Per Player in Nightterror, 1 Aug 2021",
                   labels={'x':'Troops','count':'# Players'})
fig.show()

In [62]:
df.describe()

,Troops
count,32.000000
mean,35461.125000
std,14426.240558
min,12275.000000
25%,26509.250000
50%,31895.000000
75%,43827.250000
max,66223.000000


# Notes


1.   Annotated heatmap: https://plotly.com/python/annotated-heatmap/
2.   A-Star search on grid in python at https://github.com/BaijayantaRoy/Medium-Article/blob/master/A_Star.ipynb
1. Add payout report for multiple tiles

